In [ ]:
! pip install langchain
from sklearn.feature_extraction.text import TfidfVectorizer
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer, pipeline
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain import PromptTemplate
import transformers
import requests
import numpy as np
import nltk
import re
import torch
import os
import getpass
import spacy
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
transformers.logging.set_verbosity_warning()

nltk.download('punkt')
nltk.download('stopwords')


Lemmatizer = spacy.load("en_core_web_sm")

In [3]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass.getpass("Place your token here: ")
Template = """given the Topic below, generate text about this topic in details.

Current conversation:
{history}

Topic:
{input}

Response: """

PROMPT = PromptTemplate(
                        template=Template,
                        input_variables=["history", "input"])

Place your token here:  ·····································


In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    model_id = "/kaggle/input/vicuna/pytorch/7b-v1pt5/1"
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(model_id, temperature=1.0, torch_dtype = torch.float16, device_map = "auto", offload_folder = "./cache")
    streamer = TextStreamer(tokenizer, skip_prompt=True)
    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512,streamer=streamer, return_full_text=False)
    llm = HuggingFacePipeline(pipeline=pipe)

In [5]:
generation = ConversationChain(
    llm=llm,
    prompt=PROMPT,
    verbose=False, 
    memory=ConversationBufferMemory(ai_prefix="Response")
)

In [6]:
doc1 = generation("cars")['response']
doc2 = generation("shopping")['response']
doc3 = generation("anime")['response']
Docs = [doc1, doc2, doc3]

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(



I love cars. They are a symbol of freedom and independence. They come in all shapes and sizes, from sleek sports cars to rugged off-road vehicles. Some people collect cars as a hobby, while others use them for practical purposes like transportation or work. Cars have evolved over the years, with new technologies and features being added all the time. For example, today's cars are more fuel-efficient and environmentally friendly than ever before. They also have advanced safety features like lane departure warning and automatic emergency braking. As someone who loves cars, I am always excited to see what new innovations will be introduced in the future.</s>


/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(



Shopping is a fun and exciting experience for many people. It allows us to explore new products and brands, and to find items that we need or want for our lives. Whether we are shopping in person or online, there are endless options available to us. From clothing and accessories to electronics and home goods, the possibilities are endless. Some people enjoy shopping as a leisure activity, while others use it as a practical way to get the things they need. No matter what our reasons for shopping are, it is always a great way to treat ourselves or to find unique gifts for others.</s>


/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(



Anime is a genre of Japanese animation that has gained popularity around the world. It is known for its unique style, which often features exaggerated facial expressions, colorful graphics, and fantastical themes. Many anime series are based on manga, or Japanese comics, and often explore themes of friendship, love, and personal growth. Some of the most popular anime series include "Naruto," "Dragon Ball Z," and "One Piece." In addition to television shows and movies, anime has also become popular in other forms of media, such as video games and merchandise. As someone who loves anime, I am always excited to see what new series or movies will be released in the future.</s>


In [7]:
def Clean_Text(Text):
    Cleaned_Text = re.sub(r'[^\w\s]', '', Text)
    Cleaned_Text = re.sub(r'\d+', '', Cleaned_Text)
    Cleaned_Text = Cleaned_Text.lower()
    return Cleaned_Text

def Tokenize_Text(Text):
    tokens = word_tokenize(Text)
    return tokens

def Lemmatize_Text(tokens):
    Lemmatized_Tokens = []
    for token in tokens:
        if Lemmatizer(token)[0].lemma_ != '-PRON-':
            Lemmatized_Token = Lemmatizer(token)[0].lemma_
        else:
            Lemmatized_Token = token
        Lemmatized_Tokens.append(Lemmatized_Token)
    return Lemmatized_Tokens


def Remove_Stopwords(tokens):
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token not in stop_words]
    return filtered_tokens


def Get_Unique_Words(text):
    Unique_Words = set(text)
    Sorted_Words = sorted(Unique_Words)
    return ' '.join(Sorted_Words)

In [8]:
Filtered_Docs = []
for i in range (len(Docs)):
    Cleaned_Text = Clean_Text(Docs[i])
    Tokens = Tokenize_Text(Cleaned_Text)
    Lemmatized_Tokens = Lemmatize_Text(Tokens)
    Filtered_Tokens = Remove_Stopwords(Lemmatized_Tokens)
    Unique_Words = Get_Unique_Words(Filtered_Tokens)
    Unique_Words = Unique_Words.split('\n')
    Filtered_Unique = [word for word in Unique_Words if len(word) > 2]
    Filtered_Text = ' '.join(Filtered_Unique)
    Filtered_Docs.append(Filtered_Text)

In [12]:
vectorizer = TfidfVectorizer()

tfidf_matrix = vectorizer.fit_transform(Filtered_Docs)

tfidf_matrix_dense = tfidf_matrix.toarray()

feature_names = vectorizer.get_feature_names_out()

for i, doc in enumerate(Filtered_Docs):
    print(f"TF-IDF for Document {i+1}:")
    for j, feature in enumerate(feature_names):
        tfidf_value = tfidf_matrix_dense[i, j]
        if tfidf_value > 0:
            print(f"{feature}: {tfidf_value:.4f}")
    print()

TF-IDF for Document 1:
add: 0.1483
advance: 0.1483
also: 0.1128
always: 0.0876
automatic: 0.1483
brake: 0.1483
car: 0.1483
collect: 0.1483
come: 0.1483
departure: 0.1483
emergency: 0.1483
environmentally: 0.1483
ever: 0.1483
evolve: 0.1483
example: 0.1483
excite: 0.1128
feature: 0.1128
freedom: 0.1483
friendly: 0.1483
fuelefficient: 0.1483
future: 0.1128
hobby: 0.1483
independence: 0.1483
innovation: 0.1483
introduce: 0.1483
lane: 0.1483
like: 0.1483
love: 0.1128
new: 0.0876
offroad: 0.1483
people: 0.1128
practical: 0.1128
purpose: 0.1483
rugged: 0.1483
safety: 0.1483
see: 0.1128
shape: 0.1483
size: 0.1483
sleek: 0.1483
someone: 0.1128
sport: 0.1483
symbol: 0.1483
technology: 0.1483
time: 0.1483
today: 0.1483
transportation: 0.1483
use: 0.1128
vehicle: 0.1483
warn: 0.1483
work: 0.1483
year: 0.1483

TF-IDF for Document 2:
accessory: 0.1598
activity: 0.1598
allow: 0.1598
always: 0.0944
available: 0.1598
brand: 0.1598
clothing: 0.1598
electronic: 0.1598
endless: 0.1598
enjoy: 0.1598
excit